# **Music Listening During the Coronavirus: Personality Differences**

In [26]:
import numpy as np, pandas as pd, os, sys, itertools, re, warnings, math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings(action='ignore')
LW = 0
if LW:
    os.chdir("/Users/home/Desktop/Research/Coronavirus/Public GitHub")
%run Coronavirus_Music_Functions.ipynb

In [27]:
if LW: 
    with open("/Users/home/Desktop/Research/Coronavirus/Public GitHub/factors_all_data.csv", encoding="utf8", errors='ignore') as file:
        all_data = pd.read_csv(file)
else:
    all_data = pd.read_csv('factors_all_data.csv') 
all_data.rename(columns={'Demographics_General_Gender': 'Gender'}, inplace=True)
all_data.shape

(5113, 239)

In [28]:
# reorder columns
id_col = all_data.pop('ID')
country_col = all_data.pop('Country')
gender_col = all_data.pop('Gender')

all_data.insert(0, 'Gender', gender_col)
all_data.insert(0, 'Country', country_col)
all_data.insert(0, 'ID', id_col)

In [29]:
# only keep people in listening and both groups
data = all_data[all_data['Survey Info_Group'].isin(['both','listening only'])].reset_index(drop=True)
data['Survey Info_Group'].value_counts()

both              3003
listening only    1669
Name: Survey Info_Group, dtype: int64

In [30]:
# get separate dataframes for country and factor scores
country = pd.get_dummies(data['Country'], prefix='Country')
factors = data[['F1','F2','F3','F4','F5']]

In [31]:
# only keep relevant columns
drop_cols = ['F1','F2','F3','F4','F5', 'Making Music', 'Formats', 'Demographics','Survey Info_Group','Country']
col_list = [string for string in data.columns if not any(substring in string for substring in drop_cols)]
data = data[col_list]

data.columns = data.columns.str.replace("Music Listening_", "")

In [32]:
# final df
df = pd.concat([factors, country, data], axis = 1, sort = False).reset_index(drop = True)
# df.drop(['Music Engagement_Engaging With Music Differently YN'], axis=1, inplace = True)

df.shape

(4672, 103)

#### **Factor DFs**

In [33]:
# set cutoff scores for defining 'high' and 'low' scorers
high_cutoff = 75
low_cutoff = 25

f1 = split_into_high_low(df, 'F1', low_cutoff, high_cutoff)
f2 = split_into_high_low(df, 'F2', low_cutoff, high_cutoff)
f3 = split_into_high_low(df, 'F3', low_cutoff, high_cutoff)
f4 = split_into_high_low(df, 'F4', low_cutoff, high_cutoff)
f5 = split_into_high_low(df, 'F5', low_cutoff, high_cutoff)

#### **Factor 1: Negative Emotions**

In [34]:
factor1_summary = factor_summary(f1)
cols = [i for i in factor1_summary.index if not i.startswith('Activities')]
factor1_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,0.70,-0.01,5.26,1.46,4.56,1.47
Situations_Dining,0.50,0.21,4.39,1.46,3.89,1.25
Situations_Doing Nothing Else,0.66,0.05,5.07,1.45,4.41,1.40
Situations_Evening,0.62,0.10,4.70,1.45,4.08,1.36
Situations_Exercising,0.61,0.03,4.95,1.43,4.34,1.40
Situations_Housework,0.66,-0.01,5.10,1.43,4.44,1.43
Situations_Morning,0.50,0.14,4.56,1.50,4.06,1.37
Situations_Other Activities,0.72,0.04,5.04,1.42,4.33,1.39
Situations_Sexual Activity,0.47,0.25,4.35,1.37,3.89,1.12
Situations_Traveling,0.52,0.16,4.61,1.67,4.10,1.51


#### **Factor 2: Living Situation**

In [35]:
factor2_summary = factor_summary(f2)
cols = [i for i in factor2_summary.index if not i.startswith('Activities')]
factor2_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,0.73,0.35,5.09,1.51,4.35,1.16
Situations_Dining,0.09,0.43,4.02,1.37,3.93,0.94
Situations_Doing Nothing Else,0.61,0.47,4.87,1.54,4.25,1.07
Situations_Evening,0.54,0.59,4.48,1.52,3.95,0.93
Situations_Exercising,0.91,0.47,4.92,1.49,4.01,1.01
Situations_Housework,0.56,0.41,4.86,1.50,4.29,1.09
Situations_Morning,0.16,0.52,4.19,1.51,4.03,0.99
Situations_Other Activities,0.60,0.41,4.80,1.46,4.20,1.05
Situations_Sexual Activity,0.12,0.39,4.03,1.25,3.91,0.86
Situations_Traveling,0.26,0.73,4.26,1.75,4.00,1.02


#### **Factor 3: Positive Emotions**

In [36]:
factor3_summary = factor_summary(f3)
cols = [i for i in factor3_summary.index if not i.startswith('Activities')]
factor3_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,0.36,0.05,5.07,1.48,4.70,1.42
Situations_Dining,0.74,0.35,4.54,1.48,3.80,1.13
Situations_Doing Nothing Else,0.40,0.13,4.91,1.48,4.50,1.36
Situations_Evening,0.70,0.27,4.79,1.48,4.09,1.21
Situations_Exercising,0.52,0.20,4.92,1.48,4.40,1.28
Situations_Housework,0.44,0.11,4.97,1.47,4.52,1.36
Situations_Morning,0.70,0.28,4.70,1.51,4.00,1.23
Situations_Other Activities,0.40,0.15,4.89,1.47,4.49,1.32
Situations_Sexual Activity,0.60,0.53,4.45,1.48,3.85,0.96
Situations_Traveling,0.65,0.19,4.67,1.62,4.02,1.43


#### **Factor 4: Age**

In [37]:
factor4_summary = factor_summary(f4)
cols = [i for i in factor4_summary.index if not i.startswith('Activities')]
factor4_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,0.26,0.04,4.96,1.42,4.71,1.39
Situations_Dining,0.21,0.16,4.21,1.34,3.99,1.18
Situations_Doing Nothing Else,0.31,0.02,4.84,1.39,4.52,1.37
Situations_Evening,0.22,0.11,4.44,1.36,4.22,1.24
Situations_Exercising,0.37,0.07,4.80,1.38,4.42,1.31
Situations_Housework,0.41,0.09,4.88,1.42,4.48,1.33
Situations_Morning,0.34,0.13,4.41,1.38,4.07,1.25
Situations_Other Activities,0.35,0.11,4.80,1.39,4.45,1.28
Situations_Sexual Activity,0.26,0.16,4.17,1.22,3.91,1.06
Situations_Traveling,0.15,0.26,4.30,1.65,4.14,1.39


#### **Factor 5: City Type**

In [38]:
factor5_summary = factor_summary(f5)
cols = [i for i in factor5_summary.index if not i.startswith('Activities')]
factor5_summary.loc[cols,:].round(2)

,Difference_Mean,Difference_SD,High_Mean,High_SD,Low_Mean,Low_SD
Situations_Alone,-0.23,-0.20,4.64,1.28,4.87,1.49
Situations_Dining,-0.11,-0.24,3.96,1.12,4.07,1.36
Situations_Doing Nothing Else,-0.17,-0.24,4.49,1.25,4.66,1.50
Situations_Evening,-0.17,-0.20,4.15,1.19,4.32,1.39
Situations_Exercising,-0.26,-0.17,4.34,1.24,4.61,1.41
Situations_Housework,-0.22,-0.18,4.50,1.28,4.72,1.46
Situations_Morning,-0.23,-0.18,4.05,1.22,4.27,1.40
Situations_Other Activities,-0.25,-0.16,4.43,1.26,4.67,1.42
Situations_Sexual Activity,-0.16,-0.25,3.93,0.99,4.09,1.25
Situations_Traveling,-0.30,-0.18,4.02,1.39,4.33,1.57
